In [ ]:
#At this point, none of the files above need to be loaded for this to run.
#Once the files above are generated, the file sizes are small enough that these can be loaded on the fly.

def align_list_visible_to_window_t0_pair(align_list_visible):
    coord_pair = []
    for config in align_list_visible:
        coord_pair.append((config[0], config[1][0]))
    return coord_pair

def combine_align_list_visible(align_list_visible1, align_list_visible2, print_info = False):
    print(str(len(align_list_visible1)) + '*' + str(len(align_list_visible2)) + '=' 
          +str(len(align_list_visible1)*len(align_list_visible2)) + ' combinations to check')
    coord_pair_list1 = align_list_visible_to_window_t0_pair(align_list_visible1)
    if print_info:
        print(align_list_visible1, len(align_list_visible1))
        print(coord_pair_list1, len(coord_pair_list1))
    width = sum(align_list_visible1[0][1])
    if print_info:
        print('Coordinate pair list generated for first align_list_visible')
        print(coord_pair_list1)
    total = []
    for coord_pair in enumerate(coord_pair_list1):
        config1 = align_list_visible1[coord_pair[0]]
        for config2 in align_list_visible2:
            if coord_pair[1] == (config2[1][0], config2[0]):
                for s_start in config1[3]:
                    for t_start in config2[3]:
                        check_starts = 2*sum(coord_pair[1]) + s_start + t_start
                        #This is all that needs to be computed to prove that the start numbers align
                        if check_starts == width:
                            if print_info:
                                print(s_start, t_start)
                            total += [(align_list_visible1[coord_pair[0]][1:3], s_start, config2[1:3])]
                        else:
                            if print_info:
                                print('Failed check_starts test', config1, config2)
    return total


def combine_align_list_visible_write_file(s_range, s_filename_root, t_range, t_filename_root, d_opt, root_dir):
    admissible_list = []
    if align_visible_to_make_list(s_range, t_range, d_opt, filename_root = root_dir + t_filename_root) != []:
        #In order to properly function, this is required
        #Otherwise, this function could return false negatives
        print('Abort: align_list_visible files are not complete.  Missing ' + root_dir + t_filename_root)
        print('This will yield faulty results because the input is incomplete')
        return None
    if align_visible_to_make_list(t_range, s_range, d_opt, filename_root = root_dir + s_filename_root) != []:
        print('Abort: align_list_visible files are not complete.  Missing ' + root_dir + s_filename_root)
        print('This will yield faulty results because the input is incomplete')
        return None    
    for s in s_range:
        for t in t_range:
            t_align_list_visible_filename = root_dir + t_filename_root + str(t) + '_visible_' + str(s)
            try:
                with open(t_align_list_visible_filename, 'r') as file:
                    t_align_list_visible_string = file.read()
                    print(t_align_list_visible_filename + ' read')
            except:
                t_align_list_visible_string = ''
            s_align_list_visible_filename = root_dir + s_filename_root + str(s) + '_visible_' + str(t)
            try:
                with open(s_align_list_visible_filename, 'r') as file:
                    s_align_list_visible_string = file.read()
                    print(s_align_list_visible_filename + ' read')
            except:
                s_align_list_visible_string = ''
            if len(t_align_list_visible_string) < 5 or len(s_align_list_visible_string) < 5:
                print('empty list')
                admissible_list += []
            else:
                s_align_list_visible = eval(s_align_list_visible_string)
                t_align_list_visible = eval(t_align_list_visible_string)
                admissible_list += combine_align_list_visible(t_align_list_visible, s_align_list_visible)
    admissible_list_filename = root_dir + 'admissible_list'
    with open(admissible_list_filename, 'w') as file:
        file.write(str(admissible_list))
    print('admissible_list written with ' + str(len(admissible_list)) + ' elements')
    return len(admissible_list)